In [1]:
import numpy as np
import pandas as pd
import konlpy
from konlpy.tag import Mecab
import matplotlib as plt
import re
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
ref=pd.read_csv('attraction_all_translate.csv',encoding='utf-8')
ref=ref.drop(ref.columns[[6,7,8]],axis=1)
ref=ref.dropna(subset=['score'],axis=0)
ref.astype({'score':'float'})
ref.astype({'score':'int'})
ref.info()
ref[:10]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27748 entries, 0 to 27751
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   space     27748 non-null  object 
 1   type      27748 non-null  object 
 2   language  27748 non-null  object 
 3   day       27748 non-null  int64  
 4   text      27748 non-null  object 
 5   score     27748 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 1.5+ MB


,space,type,language,day,text,score
0,힐스파,체험,한국어,201702,경치 최고 찜질방에서 하루 숙박했습니다. 달맞이길 윗쪽에 위치해서 해운대 일대가 한...,5.0
1,힐스파,체험,한국어,201711,경치가 좋은 스파 너무 좋아요. 해운대 바다가 보이는 스파라서 스파하는동안 기분이 ...,5.0
2,힐스파,체험,한국어,201710,여독풀기 좋은 장소 하루 종일 여행 후 피로를 녹이기 좋은 장소. 주차도 잘 됩니다...,4.0
3,힐스파,체험,한국어,201602,"달맞이 고개의 훌륭한 목욕탕 달맞이 고개에 자리하고 있으며, 고지대에 위치한 덕분에...",5.0
4,힐스파,체험,한국어,201504,해운대 달맞이고개의 찜질방. 찜질하는 찜질방입니다. 스파도 같이 운영합니다. 방문객...,3.0
5,힐스파,체험,한국어,201303,오션뷰에서 엄청싸게 숙박 돌아가는 비행기가 다음 날 아침 편이었기 때문에 이날은 숙...,3.0
6,힐스파,체험,한국어,201206,"경치는 좋아요... 밖에서 보이는 경치는 좋은 편인데,, 사람들이 많은 탓인지.. ...",3.0
7,힐스파,체험,한국어,201202,"바다온천 베스타스파의 가장 멋진점은, 탕에 들어가면 바다에서 목욕하는 듯한 느낌이 ...",4.0
8,힐스파,체험,한국어,201201,바다위에 둥둥 아침 일찍 바다를 볼 수 있다는 기대감으로 고고... 몸을 담글 수 ...,5.0
9,힐스파,체험,한국어,201105,전면 유리 밖으로 해운대 바다랑 광안대교가 보입니다 할머니랑 엄마랑 같이 밤에 한번...,4.0


In [3]:
#language가 한국어가 아닐 수도 있는 경우 삭제
ref.drop(ref.loc[ref["language"]=="일본어"].index,inplace=True)
ref.drop(ref.loc[ref["language"]=="중국어"].index,inplace=True)
for index, row in ref.iterrows():
    ref['text'][index]=re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》;]','',ref['text'][index])

C:\Users\pthdu\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [4]:
mecab=Mecab(dicpath="C:\mecab\mecab-ko-dic")
ref['token']=ref['text'].apply(mecab.pos)
ref.head()

Exception: Install MeCab in order to use it: http://konlpy.org/en/latest/install/

In [ ]:
temp=ref
print(temp['token'][0][0][0])

In [ ]:
search_list=[['행사','장소','비교','행사장','다양','재미','멋','축제','공연','티켓','제공','도전','일반','기본','동일','평상시','유사','기존','발견','평소','줄','자극','별거','비슷','비수기','대회'],
             ['사람','방문','곳','참여','프로그램','직접','처음','독특','흥미','수집','기대','아무것','특별','차이','전형','기대치','포기','흥분','동일'],
             ['기념품','상품','종류','품질','적당','다양','가격','축제','효율','우수','선물','상점','쇼핑','브랜드','잡화','할인','기념품','쇼핑몰','협상','낭비','돈','행상','간판','비용','고가','바가지','개점','고장','노점','흥정','객','면세점','가격표','청구','제품','고객','지하상가','디자이너','사기','관광객','쇼핑몰','가짜','수입','가격','상인','거스름돈','본전','공예','모자','현금','단골','폐업','상품','강요','먹자골목','군것질','거품'],
             ['음식','종류','가격','맛','김치','설탕','맛집','레스토랑','음식','가격','포장','간식','냄새','쓰레기','빵','음식점','술집','거북','중식','바베큐','조리','갈치','주문'],
             ['생각','일정','홍보','내용','정보','목적','생각','경험','공부','곳','때','분류','혜택','관심','생각','예상','목적','광고','시기','방식','의도','검토','잠재','메리트','소문'],
             ['안내','관광','안내소','시설','자료','구비','행사','장소','책자','지도','팜플렛','표준','뜻','여행자','안내소','기사','여행객','번역','관광지','안내서','개인','배부'],
             ['행사장','안내요원','직원','사람','분위기','대응','신속','친절','사람','서비스','인상','직업','무시','화','정직','점원','요구','주인','말','무례','우호','욕설','욕','태도','위화감'],
             ['기차','버스','지하철','차','택시','도보','지하철','버스','편도','경로','근처','대중교통','산책로','보도','통행','기차역','에스컬레이터','왕래','방면','방해','승강장','혼잡','목적지','지하도','부두','국제터미널','정류장','길가','승강장','건너편','승차','가도','우회전','셔틀버스','통과','종점','자전거','지하철역'],
             ['교통','주차','시간','때','주차장','시간','역','교통','차','주차','용이','노선','고속','길거리','주차','부과','반환','보행자'],
             ['휴식공간','벤치','휴게실','편안','수퍼','편의점','가게','편의','로비','전용','수질','유모차'],
             ['시설','화장실','공중','위생','오염','화장실'],
             ['공간','곳','근처','거리','좌석','디자인','자리','거리','장소','사물함','주위','공사','설비','조형물','설치','좌우','분리'],
             ['문화','부산','해운대','지역','문화','예술','지역','전통','유명','산업','박물관','전시실'],
             ['안전','시설','안심','보호','위험','휠체어'],
             ['가치','경치','풍경','만족','호감','참여','최고','사랑','완벽','절대','가치','환상','만족','굿','자랑','여행','충전','만점','감동','평화','평균','실망','평가','보통','비교','부족','최악','유감','결과','불편','개선','의견','이유','매력','아쉬움','결론','관광','느낌','불평','의문','망설','비추','장점','슬픔'],
             ['참여','추천','추천','강추','감사','가족','친구']
            ]

temp=[]

def score_check(text_tokens,i):
    for text_token in text_tokens:
        if text_token[0] in search_list[i]:
            return 1
        else:
            continue
    return 0

In [ ]:
from tqdm import tqdm
dc=[]
dc_temp=[]
for i in tqdm(range(len(search_list))):#len(search_list)
    dc_temp=ref.copy()
    s=[]
    print(dc_temp)
    for index,row in dc_temp.iterrows():
        s.append(score_check(dc_temp['token'][index],i))
    dc_temp['score']=s
    print(dc_temp)
    dc.append(dc_temp)

In [ ]:
ref.head()

In [ ]:
#명사, 동사, 형용사, 어근, 부정사추출
def extraction(tokens):
    temp=""
    for word in tokens:
        if word[1] in['NNG','VV','VA','XR','VCN']:
            #print(word[1])
            temp+=str(word[0])+"/"+str(word[1])
            temp+=" "
    return temp

In [ ]:
for i in range(len(search_list)):
    dc[i]['token']=dc[i]['token'].apply(extraction)


# Count Vectorizer화

In [ ]:
cv_count=[]
cv=[]

for i in range(len(search_list)):
    cv.append(CountVectorizer(
        min_df=1,
        ngram_range=(1,3),
        tokenizer=lambda x:x.split()))

    cv_count.append(cv[i].fit_transform(dc[i]['token']))

In [ ]:
cv_feature=[]
for i in range(len(search_list)):
    cv_feature.append(cv[i].get_feature_names())

In [ ]:
dc[14].head()

# Logistic Regression

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid={'C':[0.001,0.01,0.1,1,10]}
grid=[]
for i in tqdm(range(len(search_list))):
    grid.append(GridSearchCV(LogisticRegression(), param_grid, cv=5))
    grid[i].fit(cv_count[i],dc[i]['score'])

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import mglearn
import matplotlib.font_manager as fm

font_location=r'C:\Windows\Fonts\H2PORM.TTF'
font_name=fm.FontProperties(fname=font_location).get_name()
matplotlib.rc('font',family=font_name)
for i in range(len(search_list)):
    mglearn.tools.visualize_coefficients(grid[i].best_estimator_.coef_,cv_feature[i], n_top_features=20)
    plt.show()
    #plt.savefig(typelist[i]+'_keyword.jpg',facecolor='#eeeeee')